# MONTHLY BILL GENERATOR

In [4]:
from datetime import datetime, timedelta
from collections import defaultdict
from calendar import monthrange
from pprint import pprint

def get_user_month_input():
    while True:
        user_input = input("Enter billing month and year (MM YYYY): ").strip()
        try:
            dt = datetime.strptime(user_input, "%m %Y")
            return dt.strftime("%Y-%m")
        except ValueError:
            print("Invalid input. Please enter month and year as MM YYYY.")

def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d").date()

def get_month_range(target_month):
    year, month = map(int, target_month.split("-"))
    start_date = datetime(year, month, 1).date()
    end_date = datetime(year, month, monthrange(year, month)[1]).date()
    return start_date, end_date

def overlap_range(start1, end1, start2, end2):
    start = max(start1, start2)
    end = min(end1, end2)
    return (start, end) if start <= end else None

def days_between(start, end):
    return (end - start).days + 1

def normalize_item(item):
    item['qty'] = int(item['qty'])
    item['rate'] = float(item['rate'])
    item['amount'] = float(item['amount'])
    item['start_date'] = parse_date(item['start_date'])
    item['stop_date'] = parse_date(item['stop_date'])
    return item

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    target_start, target_end = get_month_range(target_month)
    total_days_in_month = days_between(target_start, target_end)
    grouped_data = defaultdict(lambda: {'qty': 0, 'amount': 0.0})

    for raw_item in item_list:
        item = normalize_item(raw_item)
        overlap = overlap_range(item['start_date'], item['stop_date'], target_start, target_end)

        if not overlap:
            continue

        active_days = days_between(overlap[0], overlap[1])
        prorated_amount = (item['rate'] * item['qty']) * (active_days / total_days_in_month)

        billing_period = f"{overlap[0]} to {overlap[1]}"
        group_key = (item['item_code'], item['rate'], billing_period)

        grouped_data[group_key]['qty'] += item['qty']
        grouped_data[group_key]['amount'] += prorated_amount

    line_items = []
    total_revenue = 0.0

    for (item_code, rate, billing_period), data in grouped_data.items():
        amount = round(data['amount'], 2)
        total_revenue += amount
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": data['qty'],
            "amount": amount,
            "billing_period": billing_period
        })

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }

item_list = [
         {
            'idx': 1,
            'item_code': 'Executive Desk (4*2)',
            'sales_description': 'Dedicated Executive Desk',
            'qty': 10,
            'rate': '1000',
            'amount': '10000',
            'start_date': '2023-11-01',
            'stop_date': '2024-10-17',
        },
        {
            'idx': 2,
            'item_code': 'Executive Desk (4*2)',
            'qty': '10',
            'rate': '1080',
            'amount': '10800',
            'start_date': '2024-10-18',
            'stop_date': '2025-10-31',
        },
        {
            'idx': 3,
            'item_code': 'Executive Desk (4*2)',
            'qty': 15,
            'rate': '1080',
            'amount': '16200',
            'start_date': '2024-11-01',
            'stop_date': '2025-10-31',
        },
        {
            'idx': 4,
            'item_code': 'Executive Desk (4*2)',
            'qty': 5,
            'rate': '1000',
            'amount': '5000',
            'start_date': '2024-11-01',
            'stop_date': '2025-10-31',
        },
        {
            'idx': 5,
            'item_code': 'Manager Cabin',
            'qty': 5,
            'rate': 5000,
            'amount': 25000,
            'start_date': '2024-11-01',
            'stop_date': '2025-10-31',
        },
        {
            'idx': 6,
            'item_code': 'Manager Cabin',
            'qty': 7,
            'rate': '5000',
            'amount': 35000,
            'start_date': '2024-12-15',
            'stop_date': '2025-10-31',
        },
        {
            'idx': 7,
            'item_code': 'Manager Cabin',
            'qty': 10,
            'rate': 4600,
            'amount': 46000,
            'start_date': '2023-11-01',
            'stop_date': '2024-10-17',
        },
        {
            'idx': 8,
            'item_code': 'Parking (2S)',
            'qty': 10,
            'rate': 1000,
            'amount': 10000,
            'start_date': '2024-11-01',
            'stop_date': '2025-10-31',
        },
        {
            'idx': 9,
            'item_code': 'Parking (2S)',
            'qty': 10,
            'rate': 0,
            'amount': 0,
            'start_date': '2024-11-01',
            'stop_date': '2025-10-31',
        },
        {
            'idx': 10,
            'item_code': 'Executive Desk (4*2)',
            'qty': '8',
            'rate': '1100',
            'amount': '8800',
            'start_date': '2024-11-15',
            'stop_date': '2025-01-31',
        },
        {
            'idx': 11,
            'item_code': 'Manager Cabin',
            'qty': '3',
            'rate': '5200',
            'amount': '15600',
            'start_date': '2024-10-10',
            'stop_date': '2024-11-10',
        },
        {
            'idx': 12,
            'item_code': 'Conference Table',
            'qty': 1,
            'rate': '20000',
            'amount': '20000',
            'start_date': '2024-11-05',
            'stop_date': '2024-11-20',
        },
        {
            'idx': 13,
            'item_code': 'Parking (2S)',
            'qty': 5,
            'rate': '1000',
            'amount': '5000',
            'start_date': '2024-11-15',
            'stop_date': '2025-02-28',
        },
        {
            'idx': 14,
            'item_code': 'Reception Desk',
            'qty': 2,
            'rate': '7000',
            'amount': '14000',
            'start_date': '2024-11-01',
            'stop_date': '2025-03-31',
        },
        {
            'idx': 15,
            'item_code': 'Reception Desk',
            'qty': 1,
            'rate': '7000',
            'amount': '7000',
            'start_date': '2024-11-10',
            'stop_date': '2024-11-25',
        },
        {
            'idx': 16,
            'item_code': 'Breakout Area',
            'qty': 3,
            'rate': '3000',
            'amount': '9000',
            'start_date': '2024-01-01',
            'stop_date': '2024-01-31',
        }
]
if __name__ == "__main__":
    target_month = get_user_month_input()
    result = generate_monthly_bill(item_list, target_month)
    pprint(result)


Enter billing month and year (MM YYYY): 01 2024
{'line_items': [{'amount': 10000.0,
                 'billing_period': '2024-01-01 to 2024-01-31',
                 'item_code': 'Executive Desk (4*2)',
                 'qty': 10,
                 'rate': 1000.0},
                {'amount': 46000.0,
                 'billing_period': '2024-01-01 to 2024-01-31',
                 'item_code': 'Manager Cabin',
                 'qty': 10,
                 'rate': 4600.0},
                {'amount': 9000.0,
                 'billing_period': '2024-01-01 to 2024-01-31',
                 'item_code': 'Breakout Area',
                 'qty': 3,
                 'rate': 3000.0}],
 'total_revenue': 65000.0}
